# Code Conversion

Prompt: 

Can you convert this Python module to C#, using the correct .Net libraries where necessary?

In [1]:
from email.mime.application import MIMEApplication
import json
import os
import urllib3
import smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
import logging

logger = logging.getLogger()
logger.setLevel(logging.INFO)

from_addr = os.environ['mailbox']
smtpserver = os.environ['smtpserver']
reply_to = os.environ['mailbox_reply_to']

def get_envs(country):
    snow_url = os.environ['snow_url_' + country]
    snow_api_url = os.environ['snow_api_url_' + country]
    client_id = os.environ['client_id_' + country]
    client_secret = os.environ['client_secret_' + country]
    username = os.environ['username_' + country]
    password = os.environ['password_' + country]
    return snow_url, snow_api_url, client_id, client_secret, username, password

def get_article_from_servicenow(country, article_number):
    if not article_number:
        return "-- Article Number Not Provided --"

    # Get the country-specific environment variables
    snow_url, snow_api_url, client_id, client_secret, username, password = get_envs(country)

    http = urllib3.PoolManager()
    url = snow_url
    headers = {
        'Content-Type': 'application/x-www-form-urlencoded'
    }
    body = f'grant_type=password&client_id={client_id}&client_secret={client_secret}&username={username}&password={password}'
    response = http.request('POST', url, headers=headers, body=body.encode('utf-8'))
    token = json.loads(response.data.decode('utf-8'))["access_token"]

    url = f"{snow_api_url}{article_number}"
    logging.info(f'Fetching article {article_number} from ServiceNow.\nURL: {url}')

    headers = {
        'Authorization': f'Bearer {token}'
    }
    try:
        response = http.request('GET', url, headers=headers)
        logging.info(f'Article {article_number} fetched successfully.\nResponse: {response}')
    except Exception as e:
        return f"-- Error fetching article {article_number} -- {e} --"
    try:
        article = json.loads(response.data.decode('utf-8'))
    except Exception as e:
        return f"-- Error parsing response for article {article_number} -- {e} --"
    if "kbBody" in article:
        article_body = article["kbBody"]
        if article_body:
            if len(article_body) > 1:
                article_text = ' '.join(article_body)
            else:
                article_text = article_body[0]
        else:
            return f"-- Article {article_number} Not Found --"
    elif "ArticleBody" in article:
        article_body = article["ArticleBody"]
        if article_body:
            if len(article_body) > 1:
                article_text = ' '.join(article_body)
            else:
                article_text = article_body[0]
        else:
            return f"-- Article {article_number} Not Found --"
    # if we get this far, we found the article text
    # we need to replace all occurences of "<span></span>" with "" to remove the empty span tags
    article_text = article_text.replace("<span></span>", "").replace("<span id=\"__caret\">_</span>", "")
    return article_text

def send_email(country, recipients, subject, header, footer, intro, article_number, attachment=None):
    if article_number and intro:
        article = get_article_from_servicenow(country, article_number)
        body = f'{intro}<br><br>{article}<p>'
    elif article_number:
        article = get_article_from_servicenow(country, article_number)
        body = f"</p>{article}<p>"
    elif intro:
        body = intro
    else:
        body = "-- Error: No Intro and No Article Number Provided --"

    to_addr = recipients
    
    # setup the MIME
    message = MIMEMultipart()
    message['From'] = from_addr
    message['To'] = to_addr
    message['Reply-To'] = reply_to
    message['Subject'] = subject
    
    # add in the message body
    message.attach(MIMEText(f'{header}{body}{footer}', 'html', 'UTF-8'))
    # message.attach(MIMEText(f'{body}', 'text', 'UTF-8'))

    # Attach the article if provided
    server = smtplib.SMTP(smtpserver, 25)
    server.starttls()
    try:
        server.sendmail(from_addr, to_addr, message.as_string())
    except Exception as e:
        raise e
    finally:
        server.quit()


def lambda_handler(event, context):
    try:
        http_method = event['requestContext']['http']['method']
        event_path = event['rawPath']
    except:
        http_method = event['httpMethod']
        event_path = event['path']
        
    try:
        body = json.loads(event['body'])
    except Exception as e:
        logging.error(f'Error loading message body.\nError: {e}\nBody:\n{event["body"]}')
        return {
            'statusCode': 500,
            'body': json.dumps({"Error": f'Error loading message body. {e}'}),
            'headers': {
                'Content-Type': 'application/json'
            }
        }
    
    # validate if the country shortcode is provided
    country = body['Country']
    if not country:
        return {
            'statusCode': 404,
            'body': json.dumps({"Error": 'Country Not Found. Use us, ca, or de.'}),
            'headers': {
                'Content-Type': 'application/json'
            }
        }    
    if country not in ['us', 'ca', 'de']:
        return {
            'statusCode': 404,
            'body': json.dumps({"Error": f'Country {country} Not Supported. Use us, ca, or de.'}),
            'headers': {
                'Content-Type': 'application/json'
            }

        }
    
    # ensure values are pulled from the event body
    try:
        recipients = body['Recipients']
        subject = body['Subject']
        header = body['Header']
        footer = body['Footer']
        if 'Intro' not in body:
            intro = ""    
        else:
            intro = body['Intro']
        if 'ArticleNumber' not in body:
            article_number = None
        else:
            article_number = body['ArticleNumber']
        logger.info(f'Article Number: {article_number}')
    except Exception as e:
        logging.error(f'Error parsing message body.\nError: {e}\nBody:\n{event["body"]}')
        return {
            'statusCode': 500,
            'body': json.dumps({"Error": f'Error parsing message body. {e}'}),
            'headers': {
                'Content-Type': 'application/json'
            }
        }
    
    #validate path and method
    if 'sendakbarticleasemail' in event_path:
        if http_method == 'POST':
            try:
                send_email(country, recipients, subject, header, footer, intro, article_number)
                return {
                    'statusCode' : 200,
                    'body': json.dumps({"Msg": "Email sent successfully."}),
                    'headers': {
                        'Content-Type': 'application/json'
                    }
                }
            except Exception as e:
                return {
                    'statusCode': 500,
                    'body': json.dumps({"Error": f'Error sending email. {e}'}),
                    'headers': {
                        'Content-Type': 'application/json'
                    }
                }
        else:
            return {
                'statusCode': 405,
                'body': json.dumps({"Error": f'{http_method} Method Not Allowed, {e}'}),
                'headers': {
                    'Content-Type': 'application/json'
                }
            }
    else:
        return {
            'statusCode': 404,
            'body': json.dumps({"Error": f'Function {event_path} Not Found, {e}'}),
            'headers': {
                'Content-Type': 'application/json'
            }
    }

KeyError: 'mailbox'